In [ ]:
#get region test notebook

#imports 
from astropy.nddata.utils import Cutout2D
from astropy.stats import SigmaClip, sigma_clipped_stats
from astropy.table import Table, QTable
from astropy.wcs import WCS
from astropy.io import fits
from pathlib import Path
import os
from os import path
from glob import glob
import sys
import re
import csv
import math
import random
import copy
import subprocess
import threading
import time
import timeit
import importlib

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize

import cv2
import imageio

from scipy.ndimage import gaussian_filter, zoom, uniform_filter, convolve
from scipy.stats import kde, lognorm, scoreatpercentile

from skimage import measure, color, data, exposure
from skimage.morphology import skeletonize, disk, ball, binary_dilation
from skimage.filters import meijering, sato, frangi, hessian, rank
from skimage.util import img_as_ubyte
from skimage.util.dtype import dtype_range
from skimage.restoration import inpaint

from reproject import reproject_exact, reproject_adaptive, reproject_interp

from photutils.background import Background2D, MedianBackground, LocalBackground, MMMBackground
from photutils.psf import CircularGaussianPRF, make_psf_model_image, PSFPhotometry, SourceGrouper
from photutils.segmentation import detect_sources, deblend_sources

import AnalysisFuncs as AF


def getRegionData(data, header, OrigData, OrigHeader):
    data_new = reprojectWrapper(data, header, OrigHeader, OrigData)
    return data_new 


def reprojectWrapper(OrigData, OrigHeader, OutputHeader, OutputData):
    """
    Reproject data from one fits file to another. Take OrigData and OrigHeader and reproject into the frame of OutputHeader and OutputData.

    Parameters:
    - OrigData: Data to reproject
    - OrigHeader: Header of data to reproject
    - OutputHeader: Header of data to reproject into
    - OutputData: Data to reproject into, used only for shape
    """

    reprojected_data, _ = reproject_interp((OrigData, OrigHeader), OutputHeader, shape_out=(np.shape(OutputData)))
    reprojected_data = cropNanBorder(reprojected_data, np.shape(OutputData))

    return reprojected_data

def cropNanBorder(image, expected_shape):
    
        """
        Crop the nan border away such that the image fits the expected shape

        Parameters: 
        -image (float): The data to crop. 2D array. 
        -expected_shape (int): the expected shape to crop the data into. 2 element tuple. 

        Returns: 
        - returns the cropped image
        """

        # Create a mask of non-NaN values
        mask = ~np.isnan(image)
        
        # Find the rows and columns with at least one non-NaN value
        non_nan_rows = np.where(mask.any(axis=1))[0]
        non_nan_cols = np.where(mask.any(axis=0))[0]

        # Use the min and max of these indices to slice the image
        cropped_image = image[non_nan_rows.min():non_nan_rows.max() + 1, non_nan_cols.min():non_nan_cols.max() + 1]
        
        # Get the current shape of the cropped image
        current_shape = cropped_image.shape
        
        # Check if the cropped image needs to be resized
        if current_shape != expected_shape:
            # Pad or crop to reach the expected shape
            padded_image = np.full(expected_shape, np.nan)  # Initialize with NaNs
            
            # If cropped_image is larger than expected_shape, trim it
            trim_rows = min(current_shape[0], expected_shape[0])
            trim_cols = min(current_shape[1], expected_shape[1])
            
            # Center the cropped image within the expected shape
            start_row = (expected_shape[0] - trim_rows) // 2
            start_col = (expected_shape[1] - trim_cols) // 2
            
            # Place the trimmed or centered cropped_image in the padded_image
            padded_image[start_row:start_row + trim_rows, start_col:start_col + trim_cols] = \
                cropped_image[:trim_rows, :trim_cols]
            
            return padded_image
        
        return cropped_image


In [15]:
fits_path_region = r"C:\Users\jhoffm72\Documents\FilPHANGS\Data\masks_v5_simple\NGC0628_simple.fits"
fits_path_orig = r"C:\Users\jhoffm72\Documents\FilPHANGS\Data\OriginalImages\ngc0628_F770W_JWST_Emission_starsub.fits"

with fits.open(fits_path_region, ignore_missing=True) as hdul:
    data = np.array(hdul[0].data)  # Assuming the image data is in the primary HDU
    header = hdul[0].header

with fits.open(fits_path_orig, ignore_missing=True) as hdul:
    OrigData = np.array(hdul[0].data)  # Assuming the image data is in the primary HDU
    OrigHeader = hdul[0].header

newData = getRegionData(data, header, OrigData, OrigHeader)

# FITS
fits_path = r'C:\Users\jhoffm72\Documents\FilPHANGS\Data\ngc0628_F770W\testRegion.fits'
hdu = fits.PrimaryHDU(newData, header= OrigHeader)
hdu.writeto(fits_path, overwrite=True)

INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please append "-SIP" to the CTYPE in the FITS header or the WCS object.

                 [astropy.wcs.wcs]


In [ ]:
Block_data = r"C:\Users\jhoffm72\Documents\FilPHANGS\Data\ngc0628_F770W\BkgSubDivRMS\ngc0628_F770W_JWST_Emission_starsub_CDDss0256pc.fits_BkgSubDivRMS.fits"
fits_path_orig = r"C:\Users\jhoffm72\Documents\FilPHANGS\Data\OriginalImages\ngc0628_F770W_JWST_Emission_starsub.fits"
fits_path_orig1 = r"C:\Users\jhoffm72\Documents\FilPHANGS\Data\ngc0628_F770W\SyntheticMap\Reconstruction_of_Original_ngc0628_F770W.fits"

def reprojectWrapper(OrigData, OrigHeader, OutputHeader, OutputData):
    """
    Reproject data from one fits file to another. Take OrigData and OrigHeader and reproject into the frame of OutputHeader and OutputData.

    Parameters:
    - OrigData: Data to reproject
    - OrigHeader: Header of data to reproject
    - OutputHeader: Header of data to reproject into
    - OutputData: Data to reproject into, used only for shape
    """

    reprojected_data, _ = reproject_interp((OrigData, OrigHeader), OutputHeader, shape_out=(np.shape(OutputData)))
    reprojected_data = cropNanBorder(reprojected_data, np.shape(OutputData))

    return reprojected_data


with fits.open(Block_data, ignore_missing=True) as hdul:
    BlockData = np.array(hdul[0].data)  # Assuming the image data is in the primary HDU
    BlockHeader = hdul[0].header


with fits.open(fits_path_orig, ignore_missing=True) as hdul:
    OrigData = np.array(hdul[0].data)  # Assuming the image data is in the primary HDU
    OrigHeader = hdul[0].header

with fits.open(fits_path_orig1, ignore_missing=True) as hdul:
    OrigData1 = np.array(hdul[0].data)  # Assuming the image data is in the primary HDU
    OrigHeader1 = hdul[0].header


newData = reprojectWrapper(OrigData, OrigHeader, BlockHeader, BlockData)
newData1 = reprojectWrapper(OrigData1, OrigHeader1,  BlockHeader, BlockData)

print(f'ratio: {np.nansum(newData)/np.nansum(OrigData)}')
print(f'ratio: {np.nansum(newData1)/np.nansum(OrigData)}')

19024353.55948555
ratio: 0.0
ratio: 0.0


Set DATE-AVG to '2022-07-17T12:01:53.586' from MJD-AVG.
Set DATE-END to '2022-07-17T12:54:46.016' from MJD-END'. [astropy.wcs.wcs]
Set OBSGEO-B to   -37.754891 from OBSGEO-[XYZ].
Set OBSGEO-H to 1738895745.206 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
